# 0 Setting

In [ ]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'
pretrained_model_name = 'model_microsoft.ckpt'

config = {
    'learning_rate': 2e-4,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,

    'n_epochs': 3000,
    'early_stop': 500,
    'device': device,
}


sector_id_list = [ # Not confirmed
    "XLK",  # Information Technology
    "XLV",  # Health Care
    "XLF",  # Financials
    "XLI",  # Industrials
    "XLY",  # Consumer Discretionary
    "XLE",  # Energy
    "XLB",  # Materials
    "XLC",  # Communication Services
    "XLU",  # Utilities
    "XLRE",  # Real Estate
    "XLP"  # Consumer Staples
]

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 1  #26

company_name = company_list[process_id]

config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_'+company_name+'.ckpt',
            # 'pretrained_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/premodel/' + pretrained_model_name,
            # 'continue_model_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model_1.ckpt'
            }

feature = [
    # X_1
    'input_ids',
    'attention_mask',
    'section_dummy',
    'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',

    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

11


## (1) Import

In [ ]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel, BertConfig

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00


In [ ]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [ ]:
df = pd.read_csv(config_2['input_path'])
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Capital Gains', 'today_return', 'today_return_cate', '1_day_return',
       '2_day_return', '3_day_return', '4_day_return', '5_day_return',
       '1_day_return_cate', '2_day_return_cate', '3_day_return_cate',
       '4_day_return_cate', '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12',
       'Ema_26', 'Macd', 'datetime', '^DJI', '^DJI_1_day_return', '^GSPC',
       '^GSPC_1_day_return', '^NDX', '^NDX_1_day_return', '^IXIC',
       '^IXIC_1_day_return', '^SOX', '^SOX_1_day_return', '^NYA',
       '^NYA_1_day_return', 'excess_return_^DJI', 'excess_return_^DJI_cate',
       'excess_return_^GSPC', 'excess_return_^GSPC_cate', 'excess_return_^NDX',
       'excess_return_^NDX_cate', 'excess_return_^IXIC',
       'excess_return_^IXIC_cate', 'excess_return_^SOX',
       'excess_return_^SOX_cate', 'excess_return_^NYA',
       'excess_return_^NYA_cate', 'input_ids', 'attention_mask',
       'section_dummy', '

In [ ]:
df = df.sort_values(by='datetime', ascending=True)
df.shape

(1070, 56)

In [ ]:
# Only contain selected features
df = df[feature]
df.columns

Index(['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy',
       'today_return', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX', '^NYA',
       'excess_return_^GSPC_cate', 'datetime'],
      dtype='object')

## (2) check nan

In [ ]:
df[df.isna().any(axis=1)]

,input_ids,attention_mask,section_dummy,publication_dummy,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA,excess_return_^GSPC_cate,datetime


In [ ]:
df.isnull().sum()

input_ids                   0
attention_mask              0
section_dummy               0
publication_dummy           0
today_return                0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
^NYA                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

input_ids                   0
attention_mask              0
section_dummy               0
publication_dummy           0
today_return                0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
^NYA                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

## (2) Time Period Selection

In [ ]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1006, 12)

## (3) Transform str back to list

In [ ]:
# 将字符串转换回列表的函数
def string_to_list(s):
    return ast.literal_eval(s)

# 将列中的字符串转换回列表
df['input_ids'] = df['input_ids'].apply(string_to_list)
df['attention_mask'] = df['attention_mask'].apply(string_to_list)
df['section_dummy'] = df['section_dummy'].apply(string_to_list)
df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

In [ ]:
df['input_ids'][0][0]

[101,
 2149,
 15768,
 1011,
 2813,
 2358,
 27790,
 2015,
 2044,
 5410,
 2707,
 2000,
 2095,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
df['attention_mask'][0][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
df['section_dummy'][0][0]

[0, 0, 1, 0, 0, 0, 0, 0, 0]

## (3) List: Same amount of elements

In [ ]:
input_ids_list_length = len(df['input_ids'][0][0])
attention_mask_list_length = len(df['attention_mask'][0][0])
section_dummy_list_length = len(df['section_dummy'][0][0])
publication_dummy_list_length = len(df['publication_dummy'][0][0])

In [ ]:
# 找到最大的內部列表長度
max_inner_length = max(df['input_ids'].apply(len))

# 定義一個函數來填充內部列表，使其長度達到最大值
def pad_inner_list(lst, zero_list):
    while len(lst) < max_inner_length:
        lst.append(zero_list)  # 這裡可以填充任何你想要的值，例如 None

# 將 "input_ids" 列中的每個內部列表填充到最大長度
df['input_ids'].apply(pad_inner_list, zero_list=[0] * input_ids_list_length)
df['attention_mask'].apply(pad_inner_list, zero_list=[0] * attention_mask_list_length)
df['section_dummy'].apply(pad_inner_list, zero_list=[0] * section_dummy_list_length)
df['publication_dummy'].apply(pad_inner_list, zero_list=[0] * publication_dummy_list_length)
df

,input_ids,attention_mask,section_dummy,publication_dummy,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA,excess_return_^GSPC_cate
0,"[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, ...",-0.000565,-0.014739,-0.012531,0.002854,0.001111,0.008979,-0.001845,1
1,"[[101, 2851, 11376, 1024, 2924, 3627, 4487, 23...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, ...",0.001974,0.000651,0.001455,-0.006175,-0.005370,-0.012003,0.002104,1
2,"[[101, 10651, 1016, 1011, 1040, 18279, 3567, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, ...",0.006138,-0.014475,-0.010663,0.007758,0.004570,-0.011092,-0.003948,1
3,"[[101, 1996, 10867, 2080, 8731, 2000, 4965, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, ...",-0.001013,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667,-0.010350,0
4,"[[101, 16021, 27595, 2015, 2360, 5366, 2024, 8...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",-0.020872,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801,-0.016923,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,"[[101, 10651, 1015, 1011, 18816, 4270, 2078, 7...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...",-0.001261,-0.001999,-0.000642,-0.000665,-0.000238,0.001204,-0.000562,0
1002,"[[101, 1022, 3971, 2000, 2488, 22149, 1996, 42...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",-0.001940,0.002871,0.003938,0.006924,0.005817,-0.000499,0.002103,1
1003,"[[101, 2004, 6494, 10431, 19281, 1010, 21442, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, ...",-0.001842,-0.001049,-0.002220,-0.003961,-0.004735,-0.006431,-0.002340,0
1004,"[[101, 2813, 2395, 7777, 2122, 23268, 15768, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, ...",-0.005965,-0.006722,-0.005802,-0.006463,-0.006492,-0.006593,-0.005384,0


In [ ]:
df['section_dummy'][0]

[[0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# 使用 apply 方法計算每個列表中元素的數量
# Note: This number is not about tokens. It's about the number of news in that day.
temp = df['input_ids'].apply(len)

# 打印 DataFrame
temp

0       9
1       9
2       9
3       9
4       9
       ..
1001    9
1002    9
1003    9
1004    9
1005    9
Name: input_ids, Length: 1006, dtype: int64

In [ ]:
# 計算"input_ids"列中所有list的平均長度
average_length = df['input_ids'].apply(len).mean()

# 計算"input_ids"列中最長的list的長度
max_length = df['input_ids'].apply(len).max()

# 計算"input_ids"列中最短的list的長度
min_length = df['input_ids'].apply(len).min()

# 打印結果
print(f"平均長度: {average_length}")
print(f"最長長度: {max_length}")
print(f"最短長度: {min_length}")


平均長度: 9.0
最長長度: 9
最短長度: 9


## int to float (section, publication)

In [ ]:
type(df.input_ids[0][0])

list

In [ ]:
def recursive_convert_to_float(item):
    if isinstance(item, list):
        return [recursive_convert_to_float(x) if x is not None else None for x in item]
    else:
        return float(item) if item is not None else None

# 使用 apply 方法將函數應用於每個元素
df['section_dummy'] = df['section_dummy'].apply(recursive_convert_to_float)
df['publication_dummy'] = df['publication_dummy'].apply(recursive_convert_to_float)

In [ ]:
df['section_dummy'][0]

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

## (4) Train_test_split

In [ ]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']

In [ ]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1006, 11)
y: (1006,)


In [ ]:
# 2. train_test_split
# val dataset for final examination
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=config['seed'], shuffle=config['shuffle'])
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.66, random_state=config['seed'], shuffle=config['shuffle'])
X_train


,input_ids,attention_mask,section_dummy,publication_dummy,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA
0,"[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",-0.000565,-0.014739,-0.012531,0.002854,0.001111,0.008979,-0.001845
1,"[[101, 2851, 11376, 1024, 2924, 3627, 4487, 23...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.001974,0.000651,0.001455,-0.006175,-0.005370,-0.012003,0.002104
2,"[[101, 10651, 1016, 1011, 1040, 18279, 3567, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.006138,-0.014475,-0.010663,0.007758,0.004570,-0.011092,-0.003948
3,"[[101, 1996, 10867, 2080, 8731, 2000, 4965, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",-0.001013,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667,-0.010350
4,"[[101, 16021, 27595, 2015, 2360, 5366, 2024, 8...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",-0.020872,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801,-0.016923
...,...,...,...,...,...,...,...,...,...,...,...
598,"[[101, 1047, 2475, 7168, 2004, 12731, 14277, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.001806,0.000298,-0.001612,-0.001512,-0.001358,-0.003253,-0.001389
599,"[[101, 4766, 1011, 14765, 1005, 1055, 21368, 5...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",-0.003709,0.005234,-0.000870,-0.002137,-0.001661,-0.004453,0.002540
600,"[[101, 9130, 5176, 5198, 2000, 4604, 1999, 152...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.003954,-0.008509,-0.005076,-0.005924,-0.005712,-0.001986,-0.005106
601,"[[101, 2966, 5080, 9338, 19960, 15312, 2594, 4...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.007491,0.005215,0.007115,0.015533,0.012397,0.015664,0.003293


## (5) Scaler

In [ ]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    # 'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    '^NYA',

    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,input_ids,attention_mask,section_dummy,publication_dummy,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX,^NYA
0,"[[101, 2149, 15768, 1011, 2813, 2358, 27790, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.521230,0.355624,0.377236,0.525722,0.515721,0.591670,0.548185
1,"[[101, 2851, 11376, 1024, 2924, 3627, 4487, 23...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.555170,0.557596,0.578553,0.424253,0.436452,0.374338,0.606119
2,"[[101, 10651, 1016, 1011, 1040, 18279, 3567, 2...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.610833,0.359087,0.404126,0.580837,0.558035,0.383766,0.517342
3,"[[101, 1996, 10867, 2080, 8731, 2000, 4965, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.515249,0.258224,0.251419,0.381444,0.380842,0.388167,0.423407
4,"[[101, 16021, 27595, 2015, 2360, 5366, 2024, 8...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.249762,0.411837,0.380522,0.324291,0.299087,0.293563,0.326980
...,...,...,...,...,...,...,...,...,...,...,...
598,"[[101, 1047, 2475, 7168, 2004, 12731, 14277, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.552929,0.552968,0.534407,0.476651,0.485527,0.464971,0.554878
599,"[[101, 4766, 1011, 14765, 1005, 1055, 21368, 5...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...",0.479209,0.617738,0.545086,0.469628,0.481824,0.452541,0.612522
600,"[[101, 9130, 5176, 5198, 2000, 4604, 1999, 152...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.475922,0.437380,0.484541,0.427071,0.432268,0.478096,0.500349
601,"[[101, 2966, 5080, 9338, 19960, 15312, 2594, 4...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.628921,0.617488,0.660028,0.668223,0.653762,0.660923,0.623563


## (6) Check number

In [ ]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
0          298     0.494196                86          0.427861         105   
1          305     0.505804               115          0.572139          97   

   Test Ratio  
0    0.519802  
1    0.480198  


In [ ]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0], X_val.shape[0], X_test.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2019-12-31T00:00:00 

Sample size: 1006
Feature: Index(['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy',
       'today_return', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX', '^NYA'],
      dtype='object') 

Target: excess_return_^GSPC_cate 

Train: Val: Test = 603 201 202


# Model

## (1) Dataset & Dataloader

In [ ]:
# Dataset
X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_1
        self.input_ids = X['input_ids']
        self.attention_mask = X['attention_mask']
        self.section = X['section_dummy']
        self.publication = X['publication_dummy']

        # X_2
        self.X_2 = torch.tensor(X.drop(columns=X_1).values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.long)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_1
        input_ids_list = self.input_ids[idx : idx + self.seq_length].tolist() # All to list
        input_ids = torch.tensor(input_ids_list) # Then to tensor
        attention_mask_list = self.attention_mask[idx : idx + self.seq_length].tolist()
        attention_mask = torch.tensor(attention_mask_list)
        section_list = self.section[idx : idx + self.seq_length].tolist()
        section = torch.tensor(section_list)
        publication_list = self.publication[idx : idx + self.seq_length].tolist()
        publication = torch.tensor(publication_list)

        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return input_ids, attention_mask, section, publication, X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [ ]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], drop_last=True, pin_memory=True)

# Check loader output
for batch in train_loader:
    input_ids, attention_mask, section, publication, X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    print("Input IDs shape:", input_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("Section shape:", section.shape)
    print("Publication shape:", publication.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    # print("X_2:", X_2)
    # print("Labels:", y)

    break


Input IDs shape: torch.Size([32, 5, 9, 32])
Attention Mask shape: torch.Size([32, 5, 9, 32])
Section shape: torch.Size([32, 5, 9, 9])
Publication shape: torch.Size([32, 5, 9, 12])
X_2 shape: torch.Size([32, 5, 7])
Labels shape: torch.Size([32])


## (2) Model Architecture

### 0 Param setting

In [ ]:
new_config = {
    'h_text_size': 5,
    'h_c_size': 1,
}

config.update(new_config)

### 1 New design

In [ ]:
# New method: C as a scaler + activation function + layer normalization
class MyModel(nn.Module):
    def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.batch_size = batch_size
        self.element_size = element_size
        self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                              0, 0, 0, 0, 0, 0, 0, 0], device=device)

        self.section_length = section_length
        self.publication_length = publication_length
        self.config = config

        # 1. News
        # text
        self.base_model = base_model
        self.fc1 = nn.Linear(768, config['h_text_size'])

        # c
        self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])


        # 3. LSTM
        self.lstm_1 = nn.LSTM(config['h_text_size'] + X_2_length, 32, dropout=0.1, num_layers=2, batch_first=True, bidirectional=False)
        self.sequential = nn.Sequential(
            nn.Linear(32, 2)
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.batch_norm = nn.BatchNorm1d(config['h_text_size'])



    def forward(self, input_ids, attention_mask, section, publication, X_2):
        # 1. BERT
        flattened_input_ids = input_ids.view(-1, 32)
        flattened_attention_mask = attention_mask.view(-1, 32)
        flattened_section = section.view(-1, self.section_length)
        flattened_publication = publication.view(-1, self.publication_length)

        e_list = []
        for i in range(0, flattened_input_ids.size(0), self.element_size):
          sub_input_ids = flattened_input_ids[i:i+self.element_size]
          sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
          sub_section = flattened_section[i:i+self.element_size]
          sub_publication = flattened_publication[i:i+self.element_size]

          non_zero_mask = (sub_input_ids != 0).any(dim=1)
          non_zero_input_ids = sub_input_ids[non_zero_mask]
          non_zero_attention_mask = sub_attention_mask[non_zero_mask]
          non_zero_section = sub_section[non_zero_mask]
          non_zero_publication = sub_publication[non_zero_mask]

          # input_ids, attention_mask
          out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
          out = out.pooler_output
          h_text = self.fc1(out)

          # section, publication
          dummies = torch.cat([non_zero_section, non_zero_publication], dim=1)
          h_c = self.fc_h_c(dummies)
          activated_c = self.sigmoid(h_c)

          # text * C
          scaled_h = h_text * activated_c
          batch_norm_h = self.batch_norm(scaled_h)
          element_mean = torch.mean(batch_norm_h, dim=0)
          e_list.append(element_mean)

        temp_tensor = torch.stack(e_list)
        b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_text_size'])

        # After BERT

        h_tech = X_2

        h = torch.cat([b_tensor, h_tech], dim=2)

        # 3. LSTM
        out, _ = self.lstm_1(h)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        final_out = self.sequential(out)

        return final_out


2. With C

In [ ]:
# # New structure: With C
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         self.fc1 = nn.Linear(768, 256)
#         self.fc2 = nn.Linear(256, config['h_text_size'])

#         # c
#         self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])

#         # news (concated)
#         # self.fc_h_news = nn.Linear(config['h_text_size'] + config['h_c_size'], config['h_news_size'])
#         self.fc_h_news = nn.Linear(config['h_text_size'] + config['h_c_size'], config['h_news_size'])

#         # 2. Indicator
#         # tech
#         # self.fc_h_tech = nn.Linear(X_2_length, config['h_tech_size'])

#         # 1&2. converge
#         # news + tech
#         # self.fc_h = nn.Linear(config['h_news_size'] + X_2_length, config['h_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_news_size'] + X_2_length, 64, dropout=0.2, num_layers=2, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(64, 2)
#         )
#         self.dropout = nn.Dropout(0.2)


#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           out = out.pooler_output
#           out = self.fc1(out)
#           h_text = self.fc2(out)

#           # section, publication
#           out = torch.cat([non_zero_section, non_zero_publication], dim=1)
#           h_c = self.fc_h_c(out)

#           # h_news
#           out = torch.cat([h_text, h_c], dim=1)
#           out = self.fc_h_news(out)
#           h_news = self.dropout(out)
#           element_mean = torch.mean(h_news, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_news_size'])

#         # 2. Indicator
#         # h_tech
#         # h_tech = self.fc_h_tech(X_2)
#         h_tech = X_2

#         # h
#         h = torch.cat([b_tensor, h_tech], dim=2)
#         # h = self.fc_h(out)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


2. New method: C as a scaler

In [ ]:
# # New structure: With C
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         self.fc1 = nn.Linear(768, 128)
#         self.fc2 = nn.Linear(128, config['h_text_size'])

#         # c
#         self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])

#         # text -> news
#         self.fc_h_news = nn.Linear(config['h_text_size'], config['h_news_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_news_size'] + X_2_length, 32, dropout=0.2, num_layers=2, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(32, 2)
#         )
#         self.relu = nn.ReLU()



#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           out = out.pooler_output
#           out = self.fc1(out)
#           h_text = self.fc2(out)

#           # section, publication
#           out = torch.cat([non_zero_section, non_zero_publication], dim=1)
#           out = self.fc_h_c(out)
#           h_c = self.relu(out)

#           # text * C
#           out = h_text * h_c
#           h_news = self.fc_h_news(out)
#           element_mean = torch.mean(h_news, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_news_size'])

#         h_tech = X_2

#         h = torch.cat([b_tensor, h_tech], dim=2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


In [ ]:
# # New method: C as a scaler + activation function
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         self.fc1 = nn.Linear(768, config['h_text_size'])
#         # self.fc2 = nn.Linear(64, config['h_text_size'])

#         # c
#         self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])

#         # text -> news
#         self.fc_h_news = nn.Linear(config['h_text_size'], config['h_news_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_news_size'] + X_2_length, 64, dropout=0.2, num_layers=2, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(64, 2)
#         )
#         self.relu = nn.ReLU()
#         self.sigmoid = nn.Sigmoid()



#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           out = out.pooler_output
#           h_text = self.fc1(out)
#           # h_text = self.fc2(out)

#           # section, publication
#           out = torch.cat([non_zero_section, non_zero_publication], dim=1)
#           out = self.fc_h_c(out)
#           h_c = self.relu(out)

#           # text * C
#           out = h_text * h_c
#           out = self.fc_h_news(out)
#           h_news = self.relu(out)

#           element_mean = torch.mean(h_news, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_news_size'])

#         h_tech = X_2

#         h = torch.cat([b_tensor, h_tech], dim=2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


3. Simplified model
without c

In [ ]:
# # New structure: Save Computation
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         self.fc1 = nn.Linear(768, 128)
#         self.fc2 = nn.Linear(128, config['h_text_size'])


#         # 1&2. converge
#         # news + tech
#         # self.fc_h = nn.Linear(config['h_text_size'] + config['h_tech_size'], config['h_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_text_size']+X_2_length, 32, dropout=0.2, num_layers=3, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(32, 2)
#         )
#         self.dropout = nn.Dropout(0.2)


#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           out = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           out = out.pooler_output
#           out = self.fc1(out)
#           out = self.fc2(out)

#           element_mean = torch.mean(out, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_text_size'])

#         # 2. Indicator
#         # h_tech
#         h_tech = X_2

#         # h
#         h = torch.cat([b_tensor, h_tech], dim=2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


4. Sequence classifiction model

In [ ]:
# # New method: C as a scaler + activation function
# class MyModel(nn.Module):
#     def __init__(self, base_model, config, element_size, section_length, publication_length, X_2_length, batch_size):
#         super(MyModel, self).__init__()
#         self.seq_length = config['seq_length']
#         self.batch_size = batch_size
#         self.element_size = element_size
#         self.abandon_tensor = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#                               0, 0, 0, 0, 0, 0, 0, 0], device=device)

#         self.section_length = section_length
#         self.publication_length = publication_length
#         self.config = config

#         # 1. News
#         # text
#         self.base_model = base_model
#         # self.fc1 = nn.Linear(768, 64)
#         # self.fc2 = nn.Linear(64, config['h_text_size'])

#         # c
#         self.fc_h_c = nn.Linear(section_length + publication_length, config['h_c_size'])

#         # text -> news
#         self.fc_h_news = nn.Linear(3, config['h_news_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_news_size'] + X_2_length, 32, dropout=0.2, num_layers=2, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(32, 2)
#         )
#         self.relu = nn.ReLU()



#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # 1. News
#         flattened_input_ids = input_ids.view(-1, 32)
#         flattened_attention_mask = attention_mask.view(-1, 32)
#         flattened_section = section.view(-1, self.section_length)
#         flattened_publication = publication.view(-1, self.publication_length)

#         e_list = []
#         for i in range(0, flattened_input_ids.size(0), self.element_size):
#           # 获取当前组的子张量
#           sub_input_ids = flattened_input_ids[i:i+self.element_size]
#           sub_attention_mask = flattened_attention_mask[i:i+self.element_size]
#           sub_section = flattened_section[i:i+self.element_size]
#           sub_publication = flattened_publication[i:i+self.element_size]

#           non_zero_mask = (sub_input_ids != 0).any(dim=1)
#           non_zero_input_ids = sub_input_ids[non_zero_mask]
#           non_zero_attention_mask = sub_attention_mask[non_zero_mask]
#           non_zero_section = sub_section[non_zero_mask]
#           non_zero_publication = sub_publication[non_zero_mask]

#           # input_ids, attention_mask
#           h_text = self.base_model(input_ids=non_zero_input_ids, attention_mask=non_zero_attention_mask)
#           # h_text = out.pooler_output
#           # out = self.fc1(out)
#           # h_text = self.fc2(out)

#           # section, publication
#           out = torch.cat([non_zero_section, non_zero_publication], dim=1)
#           out = self.fc_h_c(out)
#           h_c = self.relu(out)

#           # text * C
#           out = h_text.logits * h_c
#           h_news = self.fc_h_news(out)
#           h_news = self.relu(h_news)
#           element_mean = torch.mean(h_news, dim=0)
#           e_list.append(element_mean)

#         temp_tensor = torch.stack(e_list)
#         b_tensor = temp_tensor.view(self.batch_size, self.seq_length, self.config['h_news_size'])

#         h_tech = X_2

#         h = torch.cat([b_tensor, h_tech], dim=2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

#         final_out = self.sequential(out)

#         return final_out


## (4) Load Model

### 1. Load pretrain model

In [ ]:
# 載入預訓練模型
from transformers import AutoModelForSequenceClassification
# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))
# bert_config = BertConfig(hidden_dropout_prob=0.2)
# model_name = 'ProsusAI/finbert'
model_name = 'bert-base-uncased'
base_model = BertModel.from_pretrained(model_name)
# base_model = BertModel.from_pretrained(model_name, config=bert_config)
# base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Parameter
element_size = len(df['input_ids'][0])  # 114
section_length = len(df['section_dummy'][0][0])
publication_length = len(df['publication_dummy'][0][0])
X_2_length = len(feature) - 6


### 2. Initiate Model

In [ ]:
model = MyModel(base_model, config, element_size, section_length, publication_length, X_2_length, config['batch_size'])

model.to(device)

MyModel(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

### Extra: Contunue training

In [ ]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['save_path']))
# model.to(device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

## (5) Require_grad

In [ ]:
# Freeze all layers
for param in model.base_model.parameters():
  param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.bert.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.classifier.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

base_model.embeddings.word_embeddings.weight False
base_model.embeddings.position_embeddings.weight False
base_model.embeddings.token_type_embeddings.weight False
base_model.embeddings.LayerNorm.weight False
base_model.embeddings.LayerNorm.bias False
base_model.encoder.layer.0.attention.self.query.weight False
base_model.encoder.layer.0.attention.self.query.bias False
base_model.encoder.layer.0.attention.self.key.weight False
base_model.encoder.layer.0.attention.self.key.bias False
base_model.encoder.layer.0.attention.self.value.weight False
base_model.encoder.layer.0.attention.self.value.bias False
base_model.encoder.layer.0.attention.output.dense.weight False
base_model.encoder.layer.0.attention.output.dense.bias False
base_model.encoder.layer.0.attention.output.LayerNorm.weight False
base_model.encoder.layer.0.attention.output.LayerNorm.bias False
base_model.encoder.layer.0.intermediate.dense.weight False
base_model.encoder.layer.0.intermediate.dense.bias False
base_model.encoder.la

# Training

New trainer: save based on acc

In [ ]:
def trainer(model, train_loader, val_loader, config, device):

    criterion = config['criterion']

    # ----------------------Learning Rate-----------------------
    # optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

    # learning_rates = {
    #     'base_model': 1e-5,  # 设置base_model的学习率
    #     'base_model_fc': 5e-4,
    #     'other_params': 5e-4  # 设置其他参数的学习率
    # }

    # param_groups = [
    #     {'params': model.base_model.parameters(), 'lr': learning_rates['base_model']},
    #     {'params': model.fc1.weight, 'lr': learning_rates['base_model_fc']},
    #     {'params': model.fc1.bias, 'lr': learning_rates['base_model_fc']},
    #     # {'params': model.fc2.weight, 'lr': learning_rates['base_model_fc']},
    #     # {'params': model.fc2.bias, 'lr': learning_rates['base_model_fc']},
    #     {'params': model.fc_h_c.weight, 'lr': learning_rates['base_model_fc']},
    #     {'params': model.fc_h_c.bias, 'lr': learning_rates['base_model_fc']},
    #     # {'params': model.fc_h_news.weight, 'lr': learning_rates['base_model_fc']},
    #     # {'params': model.fc_h_news.bias, 'lr': learning_rates['base_model_fc']},
    #     {'params': model.lstm_1.weight_ih_l0, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.weight_hh_l0, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.bias_ih_l0, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.bias_hh_l0, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.weight_ih_l1, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.weight_hh_l1, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.bias_ih_l1, 'lr': learning_rates['other_params']},
    #     {'params': model.lstm_1.bias_hh_l1, 'lr': learning_rates['other_params']},
    #     {'params': model.sequential[0].weight, 'lr': learning_rates['other_params']},
    #     {'params': model.sequential[0].bias, 'lr': learning_rates['other_params']},
    # ]
    # optimizer = torch.optim.AdamW(param_groups)
    # ----------------------------------------------

    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_acc, step, early_stop_count = config['n_epochs'], 0, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for input_ids, attention_mask, section, publication, X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        pred = model(input_ids, attention_mask, section, publication, X_2)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      predicted_labels_list = []
      targets_list = []
      for input_ids, attention_mask, section, publication, X_2, y in val_loader:
          input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(input_ids, attention_mask, section, publication, X_2)
              _, predicted = torch.max(pred, 1)
              loss = criterion(pred, y)
              predicted_labels_list.extend(predicted.tolist())
              targets_list.extend(y.tolist())
              loss_record.append(loss.item())
      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if accuracy > best_acc:
          best_acc = accuracy
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_acc))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return


Old trainer

In [ ]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']

#     # ----------------------Learning Rate-----------------------
#     # optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
#     optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

#     # learning_rates = {
#     #     'base_model': 1e-5,  # 设置base_model的学习率
#     #     'base_model_fc': 1e-5,
#     #     'other_params': 1e-3  # 设置其他参数的学习率
#     # }

#     # param_groups = [
#     #     {'params': model.base_model.parameters(), 'lr': learning_rates['base_model']},
#     #     {'params': model.fc1.weight, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc1.bias, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc2.weight, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc2.bias, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc_h_c.weight, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.fc_h_c.bias, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc_h_news.weight, 'lr': learning_rates['base_model_fc']},
#     #     # {'params': model.fc_h_news.bias, 'lr': learning_rates['base_model_fc']},
#     #     {'params': model.lstm_1.weight_ih_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_hh_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_ih_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_hh_l0, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_ih_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.weight_hh_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_ih_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.lstm_1.bias_hh_l1, 'lr': learning_rates['other_params']},
#     #     {'params': model.sequential[0].weight, 'lr': learning_rates['other_params']},
#     #     {'params': model.sequential[0].bias, 'lr': learning_rates['other_params']},
#     # ]
#     # optimizer = torch.optim.AdamW(param_groups)
#     # ----------------------------------------------

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#       for input_ids, attention_mask, section, publication, X_2, y in train_loader:
#         optimizer.zero_grad()  # Set gradient to zero

#         # Forward pass
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         pred = model(input_ids, attention_mask, section, publication, X_2)
#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       # 2. Evaluation
#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       predicted_labels_list = []
#       targets_list = []
#       for input_ids, attention_mask, section, publication, X_2, y in val_loader:
#           input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(input_ids, attention_mask, section, publication, X_2)
#               _, predicted = torch.max(pred, 1)
#               loss = criterion(pred, y)
#               predicted_labels_list.extend(predicted.tolist())
#               targets_list.extend(y.tolist())
#               loss_record.append(loss.item())
#       accuracy = accuracy_score(targets_list, predicted_labels_list)

#       # Mean
#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       # 3. Judge of saving model
#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


In [ ]:
# 全部訓練
trainer(model, train_loader, val_loader, config, device)


Epoch [1/3000]:   0%|          | 0/18 [00:56<?, ?it/s, loss=0.701]

Epoch [1/3000]: Train loss: 0.6936, Valid loss: 0.6887, Val Acc: 0.5729
Saving model with loss 0.573...


Epoch [2/3000]:   0%|          | 0/18 [00:52<?, ?it/s, loss=0.699]

Epoch [2/3000]: Train loss: 0.6934, Valid loss: 0.6894, Val Acc: 0.5729


Epoch [3/3000]:   0%|          | 0/18 [00:49<?, ?it/s, loss=0.698]

...

Epoch [175/3000]:   0%|          | 0/18 [00:48<?, ?it/s, loss=0.695]

Epoch [175/3000]: Train loss: 0.6930, Valid loss: 0.6921, Val Acc: 0.5625


Epoch [176/3000]:   0%|          | 0/18 [00:49<?, ?it/s, loss=0.696]

Epoch [176/3000]: Train loss: 0.6931, Valid loss: 0.6924, Val Acc: 0.5573


Epoch [177/3000]:   0%|          | 0/18 [00:05<?, ?it/s, loss=0.696]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Evaluate

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], drop_last=True, pin_memory=True)

model = MyModel(base_model, config, element_size, section_length, publication_length, X_2_length, batch_size=1)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()

accuracy_list = []

# 1. Train part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_train_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))


# 2. Val part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_val_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))

# 3. Test part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for input_ids, attention_mask, section, publication, X_2, y in con_test_loader:
        input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
        outputs = model(input_ids, attention_mask, section, publication, X_2)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))
print('macro_f1: ', f1_score(targets_list, predicted_labels_list, average='macro'))
print('weighted_f1: ', f1_score(targets_list, predicted_labels_list, average='weighted'))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)